In [63]:
# Import necessary libraries
import pandas as pd

# Load necessary data
df_events = pd.read_csv('../04.csv_clean/ufc_event_details_clean.csv')
df_fight_results = pd.read_csv('../02.csv_scrapping/ufc_fight_results.csv')
df_fighter_tott = pd.read_csv('../04.csv_clean/ufc_fighter_tott.csv')
df_fight_stats_fight_prior = pd.read_csv('../04.csv_clean/ufc_fight_stats_fight_prior.csv')

# Standardize column names
df_fight_results.columns = df_fight_results.columns.str.lower().str.strip().str.replace(' ', '_')

# Normalize 'event' columns for merging
df_fight_results['event'] = df_fight_results['event'].astype(str).str.strip()
df_events['event'] = df_events['event'].astype(str).str.strip()

# Merge 'date' from 'df_events' into the main dataframe
df_fight_results = df_fight_results.merge(df_events[['event', 'date']], left_on='event', right_on='event', how='left')

# Convert 'date' column to datetime format
df_fight_results['date'] = pd.to_datetime(df_fight_results['date'], errors='coerce')

# Filter out rows < 2016-01-01
df_fight_results = df_fight_results[df_fight_results['date'] >= '2016-01-01'].reset_index(drop=True)

# Filter out women fights
df_fight_results = df_fight_results[df_fight_results['weightclass'].str.contains('Women') == False]

df_fight_results.head(5)
#df_fight_results.head(2)
#df_events.head(5)
#df_fighter_tott.head(5)
#df_fight_stats_fight_prior.head(5)

,event,bout,outcome,weightclass,method,round,time,time_format,referee,details,url,date
0,UFC 325: Volkanovski vs. Lopes 2,Alexander Volkanovski vs. Diego Lopes,W/L,UFC Featherweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Marc Goddard,Ben Cartlidge 46 - 49.Sal D'amato 46 - 49.Clem...,http://ufcstats.com/fight-details/b06b43670238...,2026-01-31
1,UFC 325: Volkanovski vs. Lopes 2,Dan Hooker vs. Benoit Saint Denis,L/W,Lightweight Bout,KO/TKO,2,4:45,3 Rnd (5-5-5),Herb Dean,Punches to Head From Mount,http://ufcstats.com/fight-details/a7ae8f6eb5fc...,2026-01-31
2,UFC 325: Volkanovski vs. Lopes 2,Rafael Fiziev vs. Mauricio Ruffy,L/W,Lightweight Bout,KO/TKO,2,4:30,3 Rnd (5-5-5),Daniel Movahedi,Punches to Head On Ground,http://ufcstats.com/fight-details/57d7a683b85b...,2026-01-31
3,UFC 325: Volkanovski vs. Lopes 2,Tai Tuivasa vs. Tallison Teixeira,L/W,Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Marc Goddard,Ben Cartlidge 28 - 29.Sal D'amato 28 - 29.Evan...,http://ufcstats.com/fight-details/67765535bd0b...,2026-01-31
4,UFC 325: Volkanovski vs. Lopes 2,Quillan Salkilld vs. Jamie Mullarkey,W/L,Lightweight Bout,Submission,1,3:02,3 Rnd (5-5-5),Daniel Movahedi,Neck Crank From Back Control,http://ufcstats.com/fight-details/32e9bc4d0a19...,2026-01-31


In [64]:
# Extract 'round_total' from 'time_format' column
df_fight_results['round_total'] = df_fight_results['time_format'].str.extract(r'(\d+) Rnd').astype(float)

# Rename 'round' column and convert to float
df_fight_results = df_fight_results.rename(columns={'round': 'round_end'}).astype({'round_end': 'float'})

# Rename 'time' column
df_fight_results = df_fight_results.rename(columns={'time': 'round_end_time'})

# Create function to convert 'round_end_time' from object to decimal minutes
def time_to_decimal_min(time_str):
    if pd.isna(time_str) or time_str == '-':
        return 0.0
    time_str = str(time_str).strip()
    if ':' not in time_str:
        try:
            return float(time_str)
        except ValueError:
            return 0.0
    try:
        parts = time_str.split(':')
        if len(parts) == 2:
            minutes, seconds = int(parts[0]), int(parts[1])
            return round(minutes + (seconds / 60), 2)
        else:
            return float(time_str) if time_str else 0.0
    except (ValueError, IndexError):
        return 0.0

# Apply 'time_to_decimal_min' function to the 'round_end_time' column
df_fight_results['round_end_time'] = df_fight_results['round_end_time'].apply(time_to_decimal_min)

# Separate 'bout' column into 'fighter1' and 'fighter2'
df_fight_results[['fighter1', 'fighter2']] = df_fight_results['bout'].str.split(' vs. ', expand=True)

# Trim whitespace from fighter names
df_fight_results['fighter1'] = df_fight_results['fighter1'].str.strip()
df_fight_results['fighter2'] = df_fight_results['fighter2'].str.strip()

# Assign outcomes: WINNER and LOSER
def assign_outcome(row):
    if row['outcome'] == 'W/L': # Winner is FIGHTER1
        return 'Fighter1'
    elif row['outcome'] == 'L/W': # Winner is FIGHTER2
        return 'Fighter2'
    else: # Draw or No Contest
        return None

# Apply the function to assign 'outcome'
df_fight_results['outcome'] = df_fight_results.apply(assign_outcome, axis=1)


# Assign 'weightclass'
def assign_weight_class(row):
    weight = row['weightclass']
    if 'Women\'s Strawweight' in weight:
        return 'Women\'s Strawweight'
    elif 'Women\'s Flyweight' in weight:
        return 'Women\'s Flyweight'
    elif 'Women\'s Bantamweight' in weight:
        return 'Women\'s Bantamweight'  
    elif 'Women\'s Featherweight' in weight:
        return 'Women\'s Featherweight'
    if 'Flyweight' in weight:
        return 'Flyweight'
    elif 'Bantamweight' in weight:
        return 'Bantamweight'
    elif 'Featherweight' in weight:
        return 'Featherweight'
    elif 'Lightweight' in weight:
        return 'Lightweight'
    elif 'Welterweight' in weight:
        return 'Welterweight'
    elif 'Middleweight' in weight:
        return 'Middleweight'
    elif 'Light Heavyweight' in weight:
        return 'Light Heavyweight'
    elif 'Heavyweight' in weight:
        return 'Heavyweight'
    else:
        return row['weightclass']

# Apply the function 'assign_weightclass'
df_fight_results['weightclass'] = df_fight_results.apply(assign_weight_class, axis=1)

# Assign 'method': Decision/KO-TKO/Sub
def assign_method(row):
    if 'Decision' in row['method']:
        return 'Decision'
    elif 'KO/TKO' in row['method'] or 'TKO' in row['method']:
        return 'KO/TKO'
    elif 'Submission' in row['method']:
        return 'Submission'
    else:
        return 'Other'
    
# Apply the 'assign_method' function
df_fight_results['method'] = df_fight_results.apply(assign_method, axis=1)


# Calculate consecutive wins for each fighter
def assign_win_streak(row, fighter_name):
    event_date = row['date']
    
    historical_fights = df_fight_results[df_fight_results['date'] < event_date]
    
    fighter_history = historical_fights[
        (historical_fights['fighter1'] == fighter_name) | 
        (historical_fights['fighter2'] == fighter_name)
    ].sort_values('date', ascending=False)

    if fighter_history.empty:
        return 0
    
    streak = 0
    for _, fight in fighter_history.iterrows():
        won_as_f1 = (fight['fighter1'] == fighter_name) and (fight['outcome'] == 'Fighter1')
        won_as_f2 = (fight['fighter2'] == fighter_name) and (fight['outcome'] == 'Fighter2')

        if won_as_f1 or won_as_f2:
            streak += 1
            if streak == 5: # Limit reached
                break 
        else:
            # If he didn't win this fight, the consecutive streak ends immediately
            break
            
    return streak

# Apply the 'assign_win_streak' function
df_fight_results['win_streak1'] = df_fight_results.apply(lambda x: assign_win_streak(x, x['fighter1']), axis=1).astype(float)
df_fight_results['win_streak2'] = df_fight_results.apply(lambda x: assign_win_streak(x, x['fighter2']), axis=1).astype(float)

# Create 'win_streak_delta' column
df_fight_results['win_streak_delta'] = df_fight_results['win_streak1'] - df_fight_results['win_streak2']

# Merge 'dob', 'reach' and 'height' from df_fighter_tott into df_fight_results for both fighters
df_fight_results = df_fight_results.merge(df_fighter_tott[['fighter','dob', 'reach', 'height']], left_on='fighter1', right_on='fighter', how='left').drop(columns= 'fighter').rename(columns={'dob':'dob1', 'reach':'reach1', 'height':'height1'})
df_fight_results = df_fight_results.merge(df_fighter_tott[['fighter','dob', 'reach', 'height']], left_on='fighter2', right_on='fighter', how='left').drop(columns= 'fighter').rename(columns={'dob':'dob2', 'reach':'reach2', 'height':'height2'})

# Calculate 'age' from 'dob' of both fighters at the time of the fight and delta
df_fight_results['age1'] = (pd.to_datetime(df_fight_results['date']) - pd.to_datetime(df_fight_results['dob1'])).dt.days / 365.25
df_fight_results['age2'] = (pd.to_datetime(df_fight_results['date']) - pd.to_datetime(df_fight_results['dob2'])).dt.days / 365.25
df_fight_results['age_delta'] = (df_fight_results['age1'] - df_fight_results['age2'])

# Calculate 'reach' delta
df_fight_results['reach_delta'] = df_fight_results['reach1'] - df_fight_results['reach2']

# Calculate 'height' delta
df_fight_results['height_delta'] = df_fight_results['height1'] - df_fight_results['height2']

# Merge stats from 'df_fight_stats_fight_prior' into 'df_fight_results' for both fighters
stats_columns = ['fighter', 'sslpm_cum', 'sig_str_acc_%_cum', 'ssapm_cum', 'sig_str_def_%_cum', 'td_land_15m_cum', 'td_acc_%_cum', 'td_abs_15m_cum', 'td_def_%_cum', 'sub_att_15m_cum', 'sub_att_abs_15m_cum']
rename_map1 = {'sslpm_cum':'sslpm1', 'sig_str_acc_%_cum':'sig_str_acc_%1', 'ssapm_cum':'ssapm1', 'sig_str_def_%_cum':'sig_str_def_%1', 'td_land_15m_cum':'td_land_15m1', 'td_acc_%_cum':'td_acc_%1', 'td_abs_15m_cum':'td_abs_15m1', 'td_def_%_cum':'td_def_%1', 'sub_att_15m_cum':'sub_att_15m1', 'sub_att_abs_15m_cum':'sub_att_abs_15m1'}
rename_map2 = {'sslpm_cum':'sslpm2', 'sig_str_acc_%_cum':'sig_str_acc_%2', 'ssapm_cum':'ssapm2', 'sig_str_def_%_cum':'sig_str_def_%2', 'td_land_15m_cum':'td_land_15m2', 'td_acc_%_cum':'td_acc_%2', 'td_abs_15m_cum':'td_abs_15m2', 'td_def_%_cum':'td_def_%2', 'sub_att_15m_cum':'sub_att_15m2', 'sub_att_abs_15m_cum':'sub_att_abs_15m2'}

df_fight_results = df_fight_results.merge(df_fight_stats_fight_prior[stats_columns], left_on='fighter1', right_on='fighter', how='left').drop(columns='fighter').rename(columns=rename_map1)
df_fight_results = df_fight_results.merge(df_fight_stats_fight_prior[stats_columns], left_on='fighter2', right_on='fighter', how='left').drop(columns='fighter').rename(columns=rename_map2)

# Calculate 'sslpm' delta
df_fight_results['sslpm_delta'] = df_fight_results['sslpm1'] - df_fight_results['sslpm2']

# Calculate 'sig_str_acc_%_delta'
df_fight_results['sig_str_acc_%_delta'] = df_fight_results['sig_str_acc_%1'] - df_fight_results['sig_str_acc_%2']

# Calculate 'ssapm' delta
df_fight_results['ssapm_delta'] = df_fight_results['ssapm1'] - df_fight_results['ssapm2']

# Calculate 'sig_str_def_%_delta'
df_fight_results['sig_str_def_%_delta'] = df_fight_results['sig_str_def_%1'] - df_fight_results['sig_str_def_%2']  

# Calculate 'td_land_15m' delta
df_fight_results['td_land_15m_delta'] = df_fight_results['td_land_15m1'] - df_fight_results['td_land_15m2']

# Calculate 'td_acc_%' delta
df_fight_results['td_acc_%_delta'] = df_fight_results['td_acc_%1'] - df_fight_results['td_acc_%2']

# Calculate 'td_abs_15m' delta
df_fight_results['td_abs_15m_delta'] = df_fight_results['td_abs_15m1'] - df_fight_results['td_abs_15m2']

# Calculate 'td_def_%' delta
df_fight_results['td_def_%_delta'] = df_fight_results['td_def_%1'] - df_fight_results['td_def_%2']

# Calculate 'sub_att_15m' delta
df_fight_results['sub_att_15m_delta'] = df_fight_results['sub_att_15m1'] - df_fight_results['sub_att_15m2']

# Calculate 'sub_att_abs_15m' delta
df_fight_results['sub_att_abs_15m_delta'] = df_fight_results['sub_att_abs_15m1'] - df_fight_results['sub_att_abs_15m2']

# Drop unnecessary columns
df_fight_results = df_fight_results.drop(columns=['time_format', 'referee', 'details', 'url', 'win_streak1', 'win_streak2', 'dob1', 'dob2', 'age1', 'age2', 'reach1', 'reach2', 'height1', 'height2', 'sslpm1', 'sslpm2', 'sig_str_acc_%1', 'sig_str_acc_%2', 'ssapm1', 'ssapm2', 'sig_str_def_%1', 'sig_str_def_%2', 'td_land_15m1', 'td_land_15m2', 'td_acc_%1', 'td_acc_%2', 'td_abs_15m1', 'td_abs_15m2', 'td_def_%1', 'td_def_%2', 'sub_att_15m1', 'sub_att_15m2', 'sub_att_abs_15m1', 'sub_att_abs_15m2'])

# Sort columns for better readability
df_fight_results = df_fight_results[['date', 'event', 'weightclass', 'bout', 'fighter1', 'fighter2', 'outcome', 'method', 'round_end', 'round_end_time', 'round_total', 'age_delta', 'height_delta', 'reach_delta', 'win_streak_delta', 'sslpm_delta', 'sig_str_acc_%_delta', 'ssapm_delta', 'sig_str_def_%_delta', 'td_land_15m_delta', 'td_acc_%_delta', 'td_abs_15m_delta', 'td_def_%_delta', 'sub_att_15m_delta', 'sub_att_abs_15m_delta']]

# Save the cleaned dataframe to a new CSV file
df_fight_results.to_csv('../04.csv_clean/ufc_fight_results.csv', index=False)

# Display the first few rows of the updated dataframe
df_fight_results.head(5)
#df_fight_results['weightclass'].unique()
#df_fight_results.dtypes

,date,event,weightclass,bout,fighter1,fighter2,outcome,method,round_end,round_end_time,...,sslpm_delta,sig_str_acc_%_delta,ssapm_delta,sig_str_def_%_delta,td_land_15m_delta,td_acc_%_delta,td_abs_15m_delta,td_def_%_delta,sub_att_15m_delta,sub_att_abs_15m_delta
0,2026-01-31,UFC 325: Volkanovski vs. Lopes 2,Featherweight,Alexander Volkanovski vs. Diego Lopes,Alexander Volkanovski,Diego Lopes,Fighter1,Decision,5.0,5.0,...,-3.355248,0.046896,-1.120368,-0.040450,1.708954,0.241217,1.431773,0.007500,0.579175,0.881169
1,2026-01-31,UFC 325: Volkanovski vs. Lopes 2,Featherweight,Alexander Volkanovski vs. Diego Lopes,Alexander Volkanovski,Diego Lopes,Fighter1,Decision,5.0,5.0,...,-2.292593,-0.032764,-1.721284,-0.029059,1.430211,-0.538462,-0.865542,-0.093750,-0.412943,0.881921
2,2026-01-31,UFC 325: Volkanovski vs. Lopes 2,Featherweight,Alexander Volkanovski vs. Diego Lopes,Alexander Volkanovski,Diego Lopes,Fighter1,Decision,5.0,5.0,...,-2.159174,-0.144294,-1.553184,0.210626,1.561084,0.218295,-1.796781,0.161356,-0.132968,0.115948
3,2026-01-31,UFC 325: Volkanovski vs. Lopes 2,Featherweight,Alexander Volkanovski vs. Diego Lopes,Alexander Volkanovski,Diego Lopes,Fighter1,Decision,5.0,5.0,...,-2.060035,0.115265,-4.026600,0.014273,0.682016,0.205725,0.879781,0.087500,-0.297183,0.881921
4,2026-01-31,UFC 325: Volkanovski vs. Lopes 2,Featherweight,Alexander Volkanovski vs. Diego Lopes,Alexander Volkanovski,Diego Lopes,Fighter1,Decision,5.0,5.0,...,-3.223637,-0.058650,-2.638042,0.108053,1.043380,-0.138462,0.028946,0.003289,0.347793,0.161460
